In [5]:
import pandas as pd
import re

In [3]:
# Load raw Eurostat dump
df = pd.read_csv('../data/raw_demo_pjan_population.csv')
df.head()

,Unnamed: 0,freq,unit,age,sex,geo\TIME_PERIOD,1960_value,1960_flag,1961_value,1961_flag,...,2020_value,2020_flag,2021_value,2021_flag,2022_value,2022_flag,2023_value,2023_flag,2024_value,2024_flag
0,0,A,NR,TOTAL,F,AD,NaN,:,NaN,:,...,NaN,:,NaN,:,NaN,:,NaN,:,NaN,:
1,1,A,NR,TOTAL,F,AL,NaN,:,NaN,:,...,1425342.0,NaN,1419759.0,NaN,1406532.0,NaN,1394864.0,NaN,NaN,:
2,2,A,NR,TOTAL,F,AM,NaN,:,NaN,:,...,1562689.0,NaN,1565144.0,NaN,NaN,:,1571757.0,NaN,1577675.0,NaN
3,3,A,NR,TOTAL,F,AT,3757167.0,NaN,3773097.0,NaN,...,4522292.0,NaN,4535712.0,NaN,4553444.0,NaN,4619957.0,NaN,4643918.0,NaN
4,4,A,NR,TOTAL,F,AZ,NaN,:,NaN,:,...,5039100.0,NaN,5065288.0,NaN,5081846.0,NaN,5087516.0,NaN,5113564.0,NaN


In [ ]:
print("Columns:", df.columns.tolist())
print("Index names:", df.index.names)

Columns: ['Unnamed: 0', 'freq', 'unit', 'age', 'sex', 'geo\\TIME_PERIOD', '1960_value', '1960_flag', '1961_value', '1961_flag', '1962_value', '1962_flag', '1963_value', '1963_flag', '1964_value', '1964_flag', '1965_value', '1965_flag', '1966_value', '1966_flag', '1967_value', '1967_flag', '1968_value', '1968_flag', '1969_value', '1969_flag', '1970_value', '1970_flag', '1971_value', '1971_flag', '1972_value', '1972_flag', '1973_value', '1973_flag', '1974_value', '1974_flag', '1975_value', '1975_flag', '1976_value', '1976_flag', '1977_value', '1977_flag', '1978_value', '1978_flag', '1979_value', '1979_flag', '1980_value', '1980_flag', '1981_value', '1981_flag', '1982_value', '1982_flag', '1983_value', '1983_flag', '1984_value', '1984_flag', '1985_value', '1985_flag', '1986_value', '1986_flag', '1987_value', '1987_flag', '1988_value', '1988_flag', '1989_value', '1989_flag', '1990_value', '1990_flag', '1991_value', '1991_flag', '1992_value', '1992_flag', '1993_value', '1993_flag', '1994_va

In [8]:
df = df.rename(columns={'geo\\TIME_PERIOD': 'geo'})
print("Renamed to:", [c for c in df.columns if c == 'geo'])


Renamed to: ['geo']


In [16]:
value_cols = [col for col in df.columns if col.endswith('_value')]
print("Year-value columns:", value_cols[:5], "…", len(value_cols), "total")


Year-value columns: ['1960_value', '1961_value', '1962_value', '1963_value', '1964_value'] … 65 total


In [21]:
df_long = df.melt(
    id_vars=['geo', 'sex', 'age'],
    value_vars=value_cols,
    var_name='year_flag',
    value_name='population'
)
# Extract year
df_long['year'] = df_long['year_flag'].str.replace('_value', '').astype(int)
df_long = df_long.drop(columns=['year_flag'])
df_long.head()


,geo,sex,age,population,year
0,AD,F,TOTAL,NaN,1960
1,AL,F,TOTAL,NaN,1960
2,AM,F,TOTAL,NaN,1960
3,AT,F,TOTAL,3757167.0,1960
4,AZ,F,TOTAL,NaN,1960


In [23]:
print("Unique sex codes:", df_long['sex'].unique())
print("Unique age codes:", df_long['age'].unique())


Unique sex codes: ['F' 'M' 'T']
Unique age codes: ['TOTAL' 'UNK' 'Y1' 'Y10' 'Y11' 'Y12' 'Y13' 'Y14' 'Y15' 'Y16' 'Y17' 'Y18'
 'Y19' 'Y2' 'Y20' 'Y21' 'Y22' 'Y23' 'Y24' 'Y25' 'Y26' 'Y27' 'Y28' 'Y29'
 'Y3' 'Y30' 'Y31' 'Y32' 'Y33' 'Y34' 'Y35' 'Y36' 'Y37' 'Y38' 'Y39' 'Y4'
 'Y40' 'Y41' 'Y42' 'Y43' 'Y44' 'Y45' 'Y46' 'Y47' 'Y48' 'Y49' 'Y5' 'Y50'
 'Y51' 'Y52' 'Y53' 'Y54' 'Y55' 'Y56' 'Y57' 'Y58' 'Y59' 'Y6' 'Y60' 'Y61'
 'Y62' 'Y63' 'Y64' 'Y65' 'Y66' 'Y67' 'Y68' 'Y69' 'Y7' 'Y70' 'Y71' 'Y72'
 'Y73' 'Y74' 'Y75' 'Y76' 'Y77' 'Y78' 'Y79' 'Y8' 'Y80' 'Y81' 'Y82' 'Y83'
 'Y84' 'Y85' 'Y86' 'Y87' 'Y88' 'Y89' 'Y9' 'Y90' 'Y91' 'Y92' 'Y93' 'Y94'
 'Y95' 'Y96' 'Y97' 'Y98' 'Y99' 'Y_LT1' 'Y_OPEN']


In [24]:
# Filter for total population (sex='T') and all ages (age='TOTAL')
df_long = df_long[(df_long['sex'] == 'T')   # both sexes combined
                  & (df_long['age'] == 'TOTAL')]  # all age groups combined

# Now filter for your three regions
df_long = df_long[df_long['geo'].isin(['FR', 'DE', 'EA19'])]

# Check that it’s no longer empty
print(df_long.shape)
df_long.head()



(195, 5)


,geo,sex,age,population,year
130,DE,T,TOTAL,55257088.0,1960
133,EA19,T,TOTAL,263140292.0,1960
145,FR,T,TOTAL,NaN,1960
18010,DE,T,TOTAL,55958321.0,1961
18013,EA19,T,TOTAL,NaN,1961


In [25]:
df_clean = df_long[['geo', 'year', 'population']].rename(columns={'geo':'region'})
df_clean.head()


,region,year,population
130,DE,1960,55257088.0
133,EA19,1960,263140292.0
145,FR,1960,NaN
18010,DE,1961,55958321.0
18013,EA19,1961,NaN


In [27]:
# Drop any rows where population is null
df_clean = df_clean.dropna(subset=['population'])
df_clean['population'] = df_clean['population'].astype(int)



In [30]:
df_clean.to_csv('../data/cleaned_population.csv', index=False)
print("✅ cleaned_population.csv saved")


✅ cleaned_population.csv saved
